In [ ]:
!git clone https://github.com/aivclab/detr.git /tmp/packages/detr

In [ ]:
import os
import torch
import ast
import json
import shutil
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
!pip install pycocotools
# !conda install cudatoolkit=10.1 -c pytorch
# !conda install pytorch==1.6.0 torchvision==0.7.0 cudatoolkit=10.1 -c pytorch

# Preparing Data

In [ ]:
df = pd.read_csv(r"../input/irsc-airbus-bbox-list/bbox_dictionary.csv")[:1500] #5000
df.rename(columns={"Unnamed: 0": "ImageId"}, inplace=True)
# split into training and val
train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
# for df_ in [train_df, val_df]:
def process_df_to_coco_format(df_):
    df_["bbox_list"] = df_["bbox_list"].apply(ast.literal_eval)
    df_ = df_.explode("bbox_list")
    df_["width"] = df_["bbox_list"].apply(lambda x: x[3] - x[1])
    df_["height"] = df_["bbox_list"].apply(lambda x: x[2] - x[0])
    df_["top_x"] = df_["bbox_list"].apply(lambda x: x[1])
    df_["top_y"] = df_["bbox_list"].apply(lambda x: x[0])
    df_["area"] = df_["bbox_list"].apply(lambda x: (x[3] - x[1])*(x[2] - x[0]))
    return df_

train_df = process_df_to_coco_format(train_df)
val_df = process_df_to_coco_format(val_df)

In [ ]:
# check that image have same shape
sanity_test_df = train_df.tail(10).copy()
sanity_test_df["shape"] = sanity_test_df["ImageId"].apply(lambda x: cv2.imread(f"../input/airbus-ship-detection/train_v2/{x}").shape)
sanity_test_df

In [ ]:
train_df["id"] = train_df["area"].apply(lambda x: x < 400)
train_df["id"].value_counts()

In [ ]:
val_df["id"] = val_df["area"].apply(lambda x: x < 400)
val_df["id"].value_counts()

In [ ]:
val_df

# Create directory to follow coco format

In [ ]:
# shutil.rmtree("airbus_coco1")

In [ ]:
if not os.path.exists("airbus_coco1"):
    os.makedirs("airbus_coco1/annotations")
    os.makedirs("airbus_coco1/train2017")
    os.makedirs("airbus_coco1/val2017")

In [ ]:
def make_coco_json_file(df_, ann_index, output_file_name):
    final_dict = {}
    img_list, ann_list = [], []
    for img_id, i in enumerate(df_["ImageId"].unique()):
        img_id = img_id + 1
        img_list.append(
            {"id": img_id, "width": 768, "height": 768, "file_name": i}
        )
        for x, bbox in df_[df_["ImageId"] == i].iterrows():
            ann_list.append(
                {"id": ann_index, "category_id": 1, "iscrowd": 0, "segmentation": [], "image_id": img_id, "area": bbox["area"], "bbox": [bbox["top_x"], bbox["top_y"], bbox["width"], bbox["height"]]}
            )
            ann_index += 1
    final_dict["info"] = {"description": "airbus_ship_dataset"}
    final_dict["licenses"] = []
    final_dict["images"] = img_list
    final_dict["annotations"] = ann_list
    final_dict["categories"] = [{"id": 1, "name": "ship"}]
    with open(output_file_name, "w") as r:
        json.dump(final_dict, r)

In [ ]:
make_coco_json_file(train_df, 1, "airbus_coco1/annotations/instances_train2017.json")
make_coco_json_file(val_df, 1, "airbus_coco1/annotations/instances_val2017.json")

In [ ]:
img_available = os.listdir("../input/airbus-ship-detection/train_v2")
def copy_img_for_training(df, input_path, output_path):
    for i in df["ImageId"].unique():
        if i in img_available:
            shutil.copy(f"{input_path}/{i}", output_path)
        else:
            print("error transferring image. image not found!")

copy_img_for_training(train_df, "../input/airbus-ship-detection/train_v2", "./airbus_coco1/train2017")
copy_img_for_training(val_df, "../input/airbus-ship-detection/train_v2", "./airbus_coco1/val2017")

In [ ]:
import torch
import torchvision
checkpoint = torch.hub.load_state_dict_from_url(
                url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
                map_location='cpu',
                check_hash=True)

# Remove class weights
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]

# SaveOGH
torch.save(checkpoint,
           'detr-r50_no-class-head.pth')

# Check image

In [ ]:
# with open(r"./airbus_coco1/annotations/instances_val2017.json", "r") as r:
#     data = json.load(r)
    
# # for i in data["categories"]:
# #     print(i)

# # print(data["images"][0])
# x = "../input/airbus-ship-detection/train_v2/" + data["images"][0]["file_name"]
# im = cv2.imread(x)
# for ann in data["annotations"]:
#     if ann["image_id"] == data["images"][0]["id"]:
#         print(ann)
#         bbox = ann["bbox"]
#         top = (int(bbox[0]), int(bbox[1]))
#         bottom = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
#         print(top, bottom)
#         cv2.rectangle(im, top, bottom, color=(255,0,0), thickness=2)
# plt.imshow(im)
# plt.show()

In [ ]:
import numpy as np
def convert_img():
    for i in os.listdir("./airbus_coco1/train2017"):
        img = cv2.imread(os.path.join("./airbus_coco1/train2017", i))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img2 = np.zeros_like(img)
        img2[:,:,0] = gray
        img2[:,:,1] = gray
        img2[:,:,2] = gray
#         print(img2)
        cv2.imwrite(os.path.join("./airbus_coco1/train2017", i), img2)
    
    for i in os.listdir("./airbus_coco1/val2017"):
        img = cv2.imread(os.path.join("./airbus_coco1/val2017", i))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img2 = np.zeros_like(img)
        img2[:,:,0] = gray
        img2[:,:,1] = gray
        img2[:,:,2] = gray
#         print(img2)
        cv2.imwrite(os.path.join("./airbus_coco1/val2017", i), img2)
# convert_img()

# Train Model

In [ ]:
# !cp -r myoutput_greyscale/log.txt ../

In [ ]:
!python /tmp/packages/detr/main.py --output_dir "./myoutput" --num_classes 2 --batch_size 4 --resume ../input/epoch100/checkpoint.pth --coco_path airbus_coco1 --epochs 150

In [ ]:
# COCO classes
CLASSES = ['N/A', 'ship']

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_greyscale = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])


# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes, output_name):
    f, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(pil_img)
    ax2.imshow(pil_img)
#     plt.figure(figsize=(8,5))
#     plt.imshow(pil_img)
#     ax = axarr[0, 1]
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax2.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color="red", linewidth=1))
        cl = p.argmax()
#         text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
#         ax.text(xmin, ymin, text, fontsize=15,  
#                 bbox=dict(facecolor='yellow', alpha=0.5))
#     plt.savefig(output_name)
    plt.show()


model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=2
                       )

checkpoint = torch.load(r'myoutput/checkpoint.pth',
                        map_location='cpu')

model.load_state_dict(checkpoint['model'],
                      strict=False)

def run_model():
    for i in os.listdir(r"../input/airbus-ship-detection/test_v2")[:50]:
        img_name = os.path.join(r"../input/airbus-ship-detection/test_v2", i)
        im = Image.open(img_name)
        img = transform(im).unsqueeze(0)
        outputs = model(img)
        probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
        keep = probas.max(-1).values > 0.75

        # convert boxes from [0; 1] to image scales
        bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
        plot_results(im, probas[keep], bboxes_scaled, img_name.replace("test_images", "test_images_output1"))

run_model()